# Scratchpad 3 - Pre-train LLM

## 1. Calculate text generation loss: cross-entropy and perplexity

### 1.1 Cross-entropy loss (average negative log probability value)

The model generates logits tensor of (batch_size, seq_length, vocab_size), the last dimension can be interpreted as the mode's predicted probability of each token in the whole vacabulary of being the next token. Applying `softmax` makes them probability values from 0 to 1, sumning up to 1.

For the target tokens (the input tokens minus the first token and plus the real next token in the training data), we want to maximize their logits in the output tokens. Which means we want to
-  **maximize the target tokens' softmax-ed values in the ouput tensor to 1**. 

It's easier to do this if we first calculate their log values, because to max the probabilities to 1, then we just need to maximize their log values (which are negative numbers) to 0. 

In ML we need to define a loss function and minimize one loss value calculated from the function. So instead of maximizing muliple log values of all the batches' ouput token sequences, we define the loss to be the 
- **averaged negative log probability value** of all target tokens in the output tensor, from all batches in the output concatenated. 

This is our **cross-entropy** loss. PyTorch provides a `cross_entropy` function which combines two operations:
- apply softmax 
- calculate log of softmax
- calculate negative log-likelihood loss
- average loss across batches

To use it, we need to flatten the logits tensor and the target token IDs batch tensor (aka concat all batches):

```
loss = torch.nn.functional.cross_entropy(logits_flat, target_idx_batch_flat)
```

Let's try it out.

In [20]:
from gpt.gpt_model import GPTModel
import torch

GPT_CONFIG_124M = {
    "vocab_size": 50257,  # Vocabulary size
    "context_length": 1024,  # Context length
    "embedding_dim": 768,  # Embedding dimension
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,  # Number of layers
    "dropout_rate": 0.1,  # Dropout rate
    "qkv_bias": False,  # Query-Key-Value bias
}

model = GPTModel(GPT_CONFIG_124M)
model.eval(); # ; is used to suppress the output

In [9]:
full_text = "The quick brown fox jumps over the lazy dog"
full_text_tokens = tokenizer.encode(full_text)
print(f"Full text tokens: {full_text_tokens}")

Full text tokens: [464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290]


In [14]:
input_idx = full_text_tokens[:4]
input_text = tokenizer.decode(input_idx)
print(f"Input text: {input_text}")
input_idx_batch = torch.tensor(input_idx).unsqueeze(0)
print(f"Input idx batch tensor: {input_idx_batch}")

Input text: The quick brown fox
Input idx batch tensor: tensor([[  464,  2068,  7586, 21831]])


In [17]:
with torch.no_grad():
    logits = model(input_idx_batch)
print(f"Output logits shape: {logits.shape}")

Output logits shape: torch.Size([1, 4, 50257])


In [19]:
target_idx = full_text_tokens[1:5]
target_text = tokenizer.decode(target_idx)
print(f"Target text: {target_text}")
target_idx_batch = torch.tensor(target_idx).unsqueeze(0)
print(f"Target idx batch tensor: {target_idx_batch}")

Target text:  quick brown fox jumps
Target idx batch tensor: tensor([[ 2068,  7586, 21831, 18045]])


In [24]:
# logits is the model's output of shape (batch_size, seq_len, vocab_size) -> (batch_size * seq_len, vocab_size)
logits_flat = logits.flatten(0, 1)
# targets is the target sequence's token IDs shape (batch_size, seq_len) -> (batch_size * seq_len)
targets_idx_flat = target_idx_batch.flatten(0, 1)
# Compute the cross entropy loss
loss = torch.nn.functional.cross_entropy(logits_flat, targets_idx_flat)
print(loss)

tensor(10.7479)


With our randomly initialized GPT2 model, the cross-entropy loss is 10.7479.

### 1.2 Perplexity

**Perplexity** is also a metric that measures how well the model predicts a sequence of tokens. It is the **exponential of the average negaltive log-likelihood** of the predicted probabilities for each token in a sequence, aka
- the **exponential of cross-entropy**
  
It tells us how "surprised" the model is by the test data. It can be interpreted as how many choices the model thinks it has when predicting the next token.

$$\text{Perplexity} = \exp\left(-\frac{1}{N} \sum_{i=1}^{N} \log P(x_i)\right)$$

```
cros_entropy_loss = torch.nn.functional.cross_entropy(logits_flat, target_idx_batch_flat)
perplexity_loss = torch.exp(cros_entropy_loss)
```

- A perplexity of 1 means the model is extremely sure (assigns probability of 1 to the correct next token ID).
- A high perplexity means the model is uncertain about the next token.


In [25]:
perplexity = torch.exp(loss)
print(perplexity)

tensor(46531.8984)


Our untrained model has perplexity of 46532, with the vocabulary size 50257, it is basically random guessing.

## 2. Training data pipeline

### 2.1 Create dataset

#### 2.1.1 Get data

Let's use a small dataset first. Download the "verdict.txt".

In [4]:
import os
import requests
from tqdm import tqdm


# download file from url to destination, showing progress bar
def download_file(url, destination):
    # Create the destination directory if it doesn't exist
    os.makedirs(os.path.dirname(destination), exist_ok=True)
    # Send a GET request to download the file in streaming mode
    response = requests.get(url, stream=True)

    # Get the total file size from headers, defaulting to 0 if not present
    file_size = int(response.headers.get("content-length", 0))

    # Check if file exists and has the same size
    if os.path.exists(destination):
        file_size_local = os.path.getsize(destination)
        if file_size == file_size_local:
            print(f"File already exists and is up-to-date: {destination}")
            return

    # Define the block size for reading the file
    block_size = 1024  # 1 Kilobyte

    # Initialize the progress bar with total file size
    progress_bar_description = url.split("/")[-1]  # Extract filename from URL
    with tqdm(
        total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description
    ) as progress_bar:
        # Open the destination file in binary write mode
        with open(destination, "wb") as file:
            # Iterate over the file data in chunks
            for chunk in response.iter_content(block_size):
                progress_bar.update(len(chunk))  # Update progress bar
                file.write(chunk)  # Write the chunk to the file


In [11]:
data_folder = "data"
file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

destination = os.path.join(data_folder, file_path)
download_file(url, destination)

the-verdict.txt: 20.5kiB [00:00, 15.5MiB/s]                  


In [24]:
with open(destination, 'r', encoding='utf-8') as file:
    text_data = file.read()
print(text_data[:99])
print(text_data[200:299])
print(text_data[-99:])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
 rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would h
it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."


In [39]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

total_tokens = tokenizer.encode(text_data)
print(f"Total characters: {len(text_data)}")
print(f"Total number of tokens: {len(total_tokens)}")

Total characters: 20479
Total number of tokens: 5145


#### 2.1.2 Create custom dataset

We'll create a custom DataSet which extends PyTorch's `Dataset` to prepare an LLM training dataset, which
- tokenizes the text dataset with a given tokenizer 
- chunks the text dataset into multiple chunks of max_length, with the specified stride
- using a sliding window when chunking, so that
  - input is a sequence of token ids
  - target is one token slided right of the input token id sequence

In [18]:
from torch.utils.data import Dataset
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

### 2.2 Create data loader

Then we can use this custom dataset in PyTorch's `DataLoader`, which takes care of data batching, shuffling, parallel data loading, etc.

In [33]:
from torch.utils.data import DataLoader
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

With those prepared, let's split the dataset into training and validation at ratio 9:1.

In [49]:
# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

Then let's create the dataloaders for training and validation datasets. 
- For faster training, we change the model config's context_length from 1024 to 256
- We set `drop_last=True` in the datalodaer so that if there are not enough samples left to form a full batch at the end of the dataset, we drop this smaller final batch.

In [45]:
import torch

GPT_CONFIG_124M = {
    "vocab_size": 50257,  # Vocabulary size
    "context_length": 256,  # Context length !! Let's use 256 context length for faster training
    "embedding_dim": 768,  # Embedding dimension
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,  # Number of layers
    "dropout_rate": 0.1,  # Dropout rate
    "qkv_bias": False,  # Query-Key-Value bias
}

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

Sanity check if the data is enough to cover at least context length.

In [46]:
if len(total_tokens) * (train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the training loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "increase the `training_ratio`")

if len(total_tokens) * (1-train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the validation loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "decrease the `training_ratio`")

Check that data was loaded correctly.

In [47]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


Check that token size is in the expected ballpark (we droped the last smaller batch so it is smaller than the full dataset).

In [48]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 4608
Validation tokens: 512
All tokens: 5120


### 2.3 Calculate loss batch

We implement two utils functions to calculate `cross-entropy` loss. 

One is for a given batch:

In [50]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss

The other calculates loss for multiple batches in a dataloder:

In [51]:
def calc_loss_loader(data_loader, model, device, num_batches = None):
    total_loss = 0
    if len(data_loader) == 0:
        return float('nan')
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [52]:
if torch.cuda.is_available():
   device = torch.device("cuda")
elif torch.backends.mps.is_available():
   device = torch.device("mps")
else:
   device = torch.device("cpu")

print(f"Using {device} device.")

Using mps device.


(MPS seems to get faster the second time round)

In [56]:
import time
import torch

device = "mps"

torch.manual_seed(1234)
TENSOR_A_CPU = torch.rand(5000, 5000)
TENSOR_B_CPU = torch.rand(5000, 5000)

torch.manual_seed(1234)
TENSOR_A_MPS = torch.rand(5000, 5000).to(device)
TENSOR_B_MPS = torch.rand(5000, 5000).to(device)

# Warm-up
for _ in range(100):
    torch.matmul(torch.rand(500,500).to(device), torch.rand(500,500).to(device))
    
start_time = time.time()
torch.matmul(TENSOR_A_CPU, TENSOR_B_CPU)
print("CPU : --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
torch.matmul(TENSOR_A_MPS, TENSOR_B_MPS)
print("MPS : --- %s seconds ---" % (time.time() - start_time))

CPU : --- 0.27613019943237305 seconds ---
MPS : --- 0.00039196014404296875 seconds ---


Now let's calculate the dataset's loss on an untrained model.

In [53]:
from gpt.gpt_model import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,  # Vocabulary size
    "context_length": 256,  # Context length !! Let's use 256 context length for faster training
    "embedding_dim": 768,  # Embedding dimension
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,  # Number of layers
    "dropout_rate": 0.1,  # Dropout rate
    "qkv_bias": False,  # Query-Key-Value bias
}

model = GPTModel(GPT_CONFIG_124M).to(device)

In [58]:
torch.manual_seed(123)
with torch.no_grad(): # disable gradient tracking for efficiency as we're not yet training
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)
print(f"Initial train loss: {train_loss:.16f}")
print(f"Initial validation loss: {val_loss:.16f}")

Initial train loss: 11.0054654015435105
Initial validation loss: 11.0061912536621094


## 3. Train the LLM

### 3.1 Train and eval scripts

A simple training script

In [59]:
def train_model_simple(
    model,
    train_loader,
    val_loader,
    optimizer,
    device,
    num_epochs,
    eval_freq,
    eval_iter,
    start_context,
    tokenizer,
):
    # lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # main training loop
    for epoch in range(num_epochs):
        model.train()  # set model to training mode
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()  # reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward()  # calculate gradients
            optimizer.step()  # update model weights
            tokens_seen += input_batch.numel()
            global_step += 1

            # intermediate evaluation
            if global_step % eval_freq == 0:
                model.eval()  # set model to evaluation mode
                with torch.no_grad():
                    train_loss = calc_loss_loader(
                        train_loader, model, device, eval_iter
                    )
                    val_loss = calc_loss_loader(val_loader, model, device, eval_iter)
                    train_losses.append(train_loss)
                    val_losses.append(val_loss)
                    track_tokens_seen.append(tokens_seen)
                    print(
                        f"Epoch: {epoch}, Global step: {global_step}, "
                        f"Tokens seen: {tokens_seen}, Train loss: {train_loss:.4f}, "
                        f"Validation loss: {val_loss:.4f}"
                    )
                model.train()
        # print a samplet text after each epoch
        generate_and_print_sample(model, tokenizer, device, start_context)
    return train_losses, val_losses, track_tokens_seen

Every epoch we print a text generation to see if the model is getting better. 

The `generate_and_print_sample` utility function:

In [64]:
from utils.generate_tokens import generate_tokens_greedy
import torch

def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded_tensor = torch.tensor(tokenizer.encode(start_context)).unsqueeze(0).to(device)
    with torch.no_grad():
        token_ids = generate_tokens_greedy(
            model=model,
            input_idx_batch=encoded_tensor,
            max_new_tokens=50,
            context_size=context_size,
        )
    decoded_text = tokenizer.decode(token_ids.squeeze(0).tolist())
    print(decoded_text.replace("\n", " "))
    model.train()

Now let's train the LLM using the above training script.

In [65]:
import time
import torch
from gpt.gpt_model import GPTModel

start_time = time.time()
GPT_CONFIG_124M = {
    "vocab_size": 50257,  # Vocabulary size
    "context_length": 256,  # Context length !! Let's use 256 context length for faster training
    "embedding_dim": 768,  # Embedding dimension
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,  # Number of layers
    "dropout_rate": 0.1,  # Dropout rate
    "qkv_bias": False,  # Query-Key-Value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
tokenizer = tiktoken.get_encoding("gpt2")

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model,
    train_loader,
    val_loader,
    optimizer,
    device,
    num_epochs=num_epochs,
    eval_freq=5,
    eval_iter=5,
    start_context="Every effort moves you",
    tokenizer=tokenizer,
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training took {execution_time_minutes:.2f} minutes")

Epoch: 0, Global step: 0, Tokens seen: 512, Train loss: 10.0420, Validation loss: 10.0227
Epoch: 0, Global step: 5, Tokens seen: 3072, Train loss: 8.1697, Validation loss: 8.3290
Every effort moves you,,,,, the,,,,,,, the,,,,,,,,,,,,,,,,,,,,,,,, the,,,,,,,,,,,
Epoch: 1, Global step: 10, Tokens seen: 5632, Train loss: 6.7131, Validation loss: 7.0844
Epoch: 1, Global step: 15, Tokens seen: 8192, Train loss: 6.1807, Validation loss: 6.6203
Every effort moves you, and, and, the, and, and, and, the, and, and, and, and, and, and, and, the, and, and, and, and, and, and, and,, and, the,
Epoch: 2, Global step: 20, Tokens seen: 10752, Train loss: 5.6658, Validation loss: 6.5348
Epoch: 2, Global step: 25, Tokens seen: 13312, Train loss: 5.6759, Validation loss: 6.4869
Every effort moves you .                                                 
Epoch: 3, Global step: 30, Tokens seen: 15872, Train loss: 5.2342, Validation loss: 6.4239
Epoch: 3, Global step: 35, Tokens seen: 18432, Train loss: 5.0158, 